------------------------------------------------------------------------

# WES 207 X8 FPGA Flight Controller Demo Code

### Author: Aaron Coffman

-------------------------------------------------------------------------

### Jupyter Notebook Setup for Overlay Import

In [1]:

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np
from math import *

print('')
print('done')


done


### Loading FPGA bit stream

In [3]:
# Load bitstream
#ol = Overlay('SBUS_DEMO_100M_tx_ext_clks_test5.bit')
ol = Overlay('X8_DEMO_3.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

print('')
print('done')

  
1.7901568412780762 seconds to program bitstream

done


### Setting MMIO Address Definitions

In [4]:
# Address space definitions
## MMIO targets

# Xilinx IP cores
QUAD_SPI_BASE_ADDRESS           = 0x40000000
UART_16550_BASE_ADDRESS         = 0x41C00000

# HLS IP cores
SPI_DRIVER_CTRL_BASE_ADDRESS    = 0x00000000  # placeholder
POSITION_CTRL_BASE_ADDRESS      = 0x00000000  # placeholder

UART_DRIVER_CTRL_BASE_ADDRESS   = 0x40002000

RC_RECEIVER_CTRL_BASE_ADDRESS   = 0x40003000
RC_RECEIVER_DATA_BASE_ADDRESS   = 0x40003020  # 0x20 for raw SBUS data

FLIGHT_MAIN_CTRL_BASE_ADDRESS   = 0x40004000
FLIGHT_MAIN_CMD_BASE_ADDRESS    = 0x40004010  # 0x10 for scaled SBUS data
FLIGHT_MAIN_MEAS_BASE_ADDRESS   = 0x40004020  # 0x20 for scaled Measured data

PID_CTRL_BASE_ADDRESS           = 0x40005000
PID_INPUT_CMD_BASE_ADDRESS      = 0x40005010  # 0x10 for CMD input
PID_INPUT_MEAS_BASE_ADDRESS     = 0x40005020  # 0x20 for Measured input
PID_KP_BASE_ADDRESS             = 0x40005040  # 0x40 for kp PID gains
PID_KI_BASE_ADDRESS             = 0x40005070  # 0x70 for ki PID gains
PID_KD_BASE_ADDRESS             = 0x40005060  # 0x60 for kd PID gains

PWM_CTRL_BASE_ADDRESS           = 0x40006000
PWM_INPUT_CMD_BASE_ADDRESS      = 0x40006040  # 0x40 for CMD input
PWM_MIN_DUTY_BASE_ADDRESS       = 0x40006010  # 0x10 for Min Duty Cycle input
PWM_MAX_DUTY_BASE_ADDRESS       = 0x40006018  # 0x18 for Max Duty Cycle input
PWM_PERIOD_ADDRESS              = 0x40006020  # 0x20 for Period (frequency) Input
PWM_TEST_DATA_BASE_ADDRESS      = 0x40014000


# MMIO address ranges
ADDRESS_RANGE1 = 0x1000    # 4k
ADDRESS_RANGE2 = 0x2000    # 8k
ADDRESS_RANGE3 = 0x4000    # 16k
ADDRESS_RANGE4 = 0x10000   # 64k


print('Address Definitions complete')
print('')
print('done')

Address Definitions complete

done


### Defining MMIO index offsets for read/write functions

In [5]:
# defining MMIO index offsets
index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C 
index_8 = 0x20
index_9 = 0x24
index_10 = 0x28
index_11 = 0x2C
index_12 = 0x30
index_13 = 0x34
index_14 = 0x38
index_15 = 0x3C 
index_16 = 0x40
index_17 = 0x44
index_18 = 0x48
index_19 = 0x4C
index_20 = 0x50
index_21 = 0x54
index_22 = 0x58
index_23 = 0x5C 
index_24 = 0x60
index_25 = 0x64
index_26 = 0x68
index_27 = 0x6C
index_28 = 0x70
index_29 = 0x74

print('')
print('done')


done


### Configuring PID Controller Gains

In [6]:
# calculating PID gains for 32 bit fixed point kp, ki, kd

Desired_gain = 0.200
val = Desired_gain / pow(2,-13)

print('Decimal Value: ', val   )
print('Hex Value: \t{0:x}'.format( int(val) ))


Decimal Value:  1638.4
Hex Value: 	666


In [7]:

# creating MMIO objects
kp_gains = MMIO(PID_KP_BASE_ADDRESS,ADDRESS_RANGE1, False)
ki_gains = MMIO(PID_KI_BASE_ADDRESS,ADDRESS_RANGE1, False)
kd_gains = MMIO(PID_KD_BASE_ADDRESS,ADDRESS_RANGE1, False)

# setting PID gains for DEMO/debug
kp_gains.write(index_0,0x666)   # Positon Roll  kp
kp_gains.write(index_1,0x666)   # Positon Pitch kp
kp_gains.write(index_2,0x666)   # Positon Yaw   kp
kp_gains.write(index_3,0x666)   # Rate    Roll  kp
kp_gains.write(index_4,0x666)   # Rate    Pitch kp
kp_gains.write(index_5,0x666)   # Rate    yaw   kp

ki_gains.write(index_0,0x0)   # Positon Roll   ki
ki_gains.write(index_1,0x0)   # Positon Pitch  ki
ki_gains.write(index_2,0x0)   # Rate    Roll   ki
ki_gains.write(index_3,0x0)   # Rate    Pitch  ki

kd_gains.write(index_0,0x4)   # Positon Roll   kd
kd_gains.write(index_1,0x4)   # Positon Pitch  kd
kd_gains.write(index_2,0x4)   # Rate    Roll   kd
kd_gains.write(index_3,0x4)   # Rate    Pitch  kd

# reading back PID gain values
# kp
test1 = kp_gains.read(index_0) * pow(2,-13)
test2 = kp_gains.read(index_1) * pow(2,-13)
test3 = kp_gains.read(index_2) * pow(2,-13)
test4 = kp_gains.read(index_3) * pow(2,-13)
test5 = kp_gains.read(index_4) * pow(2,-13)
test6 = kp_gains.read(index_5) * pow(2,-13)
print("kp:", test1, test2, test3, test4, test5, test6)

# ki
test1 = ki_gains.read(index_0) * pow(2,-13)
test2 = ki_gains.read(index_1) * pow(2,-13)
test3 = ki_gains.read(index_2) * pow(2,-13)
test4 = ki_gains.read(index_3) * pow(2,-13)
print("ki:", test1, test2, test3, test4)

# kd
test1 = kd_gains.read(index_0) * pow(2,-13)
test2 = kd_gains.read(index_1) * pow(2,-13)
test3 = kd_gains.read(index_2) * pow(2,-13)
test4 = kd_gains.read(index_3) * pow(2,-13)
print("kd:", test1, test2, test3, test4)


print('')
print('done')

kp: 0.199951171875 0.199951171875 0.199951171875 0.199951171875 0.199951171875 0.199951171875
ki: 0.0 0.0 0.0 0.0
kd: 0.00048828125 0.00048828125 0.00048828125 0.00048828125

done


### Configuring PWM Generator Frequency and Duty Cycle

In [8]:
# creating MMIO objects
PWM_Min_Duty = MMIO(PWM_MIN_DUTY_BASE_ADDRESS,ADDRESS_RANGE1, False)
PWM_Max_Duty = MMIO(PWM_MAX_DUTY_BASE_ADDRESS,ADDRESS_RANGE1, False)
PWM_Period = MMIO(PWM_PERIOD_ADDRESS,ADDRESS_RANGE1, False)


# setting PWM Min/Max Duty Cycle and Period for DEMO/debug
PWM_Min_Duty.write(index_0,0x3F00)   # PWM Min Duty Cycle
PWM_Max_Duty.write(index_0,0x6B20)   # PWM Max Duty Cycle
PWM_Period.write(index_0,0x8B80)     # PWM Period


# Reading PWM Min/Max Duty Cycle and Period for DEMO/debug
test1 = PWM_Min_Duty.read(index_0)
test2 = PWM_Max_Duty.read(index_0)
test3 = PWM_Period.read(index_0)
print("Min Duty:", test1,"Max Duty:", test2,"Period:", test3)



Min Duty: 16128 Max Duty: 27424 Period: 35712


### Creating MMIO Objects

In [9]:

# UART16650 Core
UART_core = MMIO(UART_16550_BASE_ADDRESS,ADDRESS_RANGE4, False)                    # 64k
UART_Driver = MMIO(UART_DRIVER_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)            # 4k

# RC Receiver Core
RC_Driver = MMIO(RC_RECEIVER_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)              # 4k
RC_CMD_Data = MMIO(RC_RECEIVER_DATA_BASE_ADDRESS,ADDRESS_RANGE1, False)            # 4k

# Flight Main Core
Flight_Main_core = MMIO(FLIGHT_MAIN_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)       # 4k
Flight_Main_CMD_Data = MMIO(FLIGHT_MAIN_CMD_BASE_ADDRESS,ADDRESS_RANGE1, False)    # 4k
Flight_Main_MEAS_Data = MMIO(FLIGHT_MAIN_MEAS_BASE_ADDRESS,ADDRESS_RANGE1, False)  # 4k

# PID Core
PID_core = MMIO(PID_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)             # 4k
PID_CMD_Data = MMIO(PID_INPUT_CMD_BASE_ADDRESS,ADDRESS_RANGE1, False)    # 4k
PID_MEAS_Data = MMIO(PID_INPUT_MEAS_BASE_ADDRESS,ADDRESS_RANGE1, False)  # 4k

# PWM Core
PWM_core = MMIO(PWM_CTRL_BASE_ADDRESS,ADDRESS_RANGE1, False)             # 4k
PWM_CMD_Data = MMIO(PWM_INPUT_CMD_BASE_ADDRESS,ADDRESS_RANGE1, False)    # 4k
PWM_Test_Data = MMIO(PWM_TEST_DATA_BASE_ADDRESS,ADDRESS_RANGE3, False)   # 16k


# UART scratch register address
SCRATCH_REG      =   0x101C 

# setting WHO_AM_I value for debug
#UART_core.write(SCRATCH_REG,0xFF)

print('UART16550 Core Configuration Complete')
print('')
print('done')

UART16550 Core Configuration Complete

done


### Starting HLS Cores

In [10]:
# NOTE, starting last core in chain first
# PWM -> PID -> FLight_Main -> RC_Receiver -> UART_Driver

# setting ap_start/auto reset on PWM core
#PWM_core.write(0x00,0x01)  # runs once
PWM_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on PID core
#PID_core.write(0x00,0x01)  # runs once
PID_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on Flight main
#Flight_Main_core.write(0x00,0x01)  # runs once
Flight_Main_core.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on RC Receiver
#RC_Driver.write(0x00,0x01)  # runs once
RC_Driver.write(0x00,0x81) # runs continuously

# setting ap_start/auto reset on UART Driver
#UART_Driver.write(0x00,0x01)  # runs once
UART_Driver.write(0x00,0x81) # runs continuously


# reading back CONFIG registers for HLS cores
CONFIG_REG5 = PWM_core.read(0x00)
CONFIG_REG4 = PID_core.read(0x00)
CONFIG_REG3 = Flight_Main_core.read(0x00)
CONFIG_REG2 = RC_Driver.read(0x00)
CONFIG_REG1 = UART_Driver.read(0x00)


print('CONFIG_REG UART: \t', CONFIG_REG1)
print('CONFIG_REG RC: \t\t', CONFIG_REG2)
print('CONFIG_REG Flight Main: ', CONFIG_REG3)
print('CONFIG_REG PID: \t', CONFIG_REG4)
print('CONFIG_REG PWM: \t', CONFIG_REG5)
print('')
print('')

CONFIG_REG UART: 	 131
CONFIG_REG RC: 		 131
CONFIG_REG Flight Main:  131
CONFIG_REG PID: 	 131
CONFIG_REG PWM: 	 131




### UART Driver Configuration Confirmation and Status

In [11]:

# UART16650 Line status register offset
LINE_STATUS_REG  =   0x1014 

# reading scratch register for configuration verification, expect 0x45
test_read = UART_core.read(SCRATCH_REG)
print('WHO_AM_I: \t', test_read)

# reading FIFO status of UART core
# Overrun Error = 1 indicates RX FIFO Overflow
OVERRUN_ERROR = UART_core.read(LINE_STATUS_REG)
OVERRUN_ERROR = OVERRUN_ERROR & 0x02 # grabbing bit 1
print('OVERRUN_ERROR: \t', OVERRUN_ERROR,'\r')


print('')
print('done')

WHO_AM_I: 	 69
OVERRUN_ERROR: 	 0 

done


In [27]:
PWM_CMD_Data

#motor outputs
TEST_READ1  = PWM_CMD_Data.read(index_0)
TEST_READ2  = PWM_CMD_Data.read(index_1)
TEST_READ3  = PWM_CMD_Data.read(index_2)
TEST_READ4 = PWM_CMD_Data.read(index_3)
TEST_READ5 = PWM_CMD_Data.read(index_4)
TEST_READ6 = PWM_CMD_Data.read(index_5)
TEST_READ7 = PWM_CMD_Data.read(index_6)
TEST_READ8 = PWM_CMD_Data.read(index_7)
TEST_READ9 = PWM_CMD_Data.read(index_8)

print('M1 out PID: \t', "%1.3F"% (TEST_READ1 * pow(2,-13)))
print('M2 out PID: \t', "%1.3F"% (TEST_READ2 * pow(2,-13)))
print('M3 out PID: \t', "%1.3F"% (TEST_READ3 * pow(2,-13)))
print('M4 out PID: \t', "%1.3F"% (TEST_READ4 * pow(2,-13)))
print('M5 out PID: \t', "%1.3F"% (TEST_READ5 * pow(2,-13)))
print('M6 out PID: \t', "%1.3F"% (TEST_READ6 * pow(2,-13)))
print('M7 out PID: \t', "%1.3F"% (TEST_READ7 * pow(2,-13)))
print('M8 out PID: \t', "%1.3F"% (TEST_READ8 * pow(2,-13)))
print('ARM State: \t', "%1d"% (TEST_READ9 * pow(2,-13)))


print('')
print('done')


M1 out PID: 	 62776.999
M2 out PID: 	 62840.999
M3 out PID: 	 62840.999
M4 out PID: 	 62776.999
M5 out PID: 	 0.000
M6 out PID: 	 0.000
M7 out PID: 	 0.000
M8 out PID: 	 0.000
ARM State: 	 0

done


# RC Receiver Data Flow to PID controller Real-Time DEMO

In [13]:
# X8 Flight Controller Demo - RC controls
from __future__ import print_function
import sys
count = 0

print('')
print('')
while(count < 10000):
    
    # reading PWM Outputs
    TEST_READ1  = PWM_Test_Data.read(index_0)
    TEST_READ2  = PWM_Test_Data.read(index_1)
    TEST_READ3  = PWM_Test_Data.read(index_2)
    TEST_READ4  = PWM_Test_Data.read(index_3)
    TEST_READ5  = PWM_Test_Data.read(index_4)
    TEST_READ6  = PWM_Test_Data.read(index_5)
    TEST_READ7  = PWM_Test_Data.read(index_6)
    TEST_READ8  = PWM_Test_Data.read(index_7)
    

    #print('')
    print('M1: ', (TEST_READ1),'M2: ', (TEST_READ2),'M3: ', (TEST_READ3),'M4: ', (TEST_READ4),'M5: ', (TEST_READ5),'M6: ', (TEST_READ6),'M7: ', (TEST_READ7),'M8: ', (TEST_READ8),'  ', end='\r')
    
    time.sleep(0.25) 
    sys.stdout.flush()
    count = count + 1
print("")

KeyboardInterrupt: 

### End of DEMO code